In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/new-york-city-taxi-fare-prediction/train.csv',encoding = 'latin', nrows=350000)

In [ ]:
df.head()

In [ ]:
df.sample(10)

In [ ]:
t = df.loc[(df['pickup_longitude']<-90) | (df['dropoff_longitude']<-90) | (df['pickup_latitude']>90) | (df['dropoff_latitude']>90)]
t.head()

In [ ]:
df = df[(df['pickup_longitude'].notna()) & (df['dropoff_longitude'].notna()) & (df['pickup_latitude'].notna()) & (df['dropoff_latitude'].notna())]

In [ ]:
df.drop(df.loc[(df['pickup_longitude']<-90) | (df['dropoff_longitude']<-90) | (df['pickup_latitude']>90) | (df['dropoff_latitude']>90) | (df['pickup_longitude']>90) | (df['dropoff_longitude']>90) | (df['pickup_latitude']<-90) | (df['dropoff_latitude']<-90)].index, inplace=True)

In [ ]:
df = df[(-76 <= df['pickup_longitude']) & (df['pickup_longitude'] <= -72) & (-76 <= df['dropoff_longitude']) & (df['dropoff_longitude'] <= -72) & (38 <= df['pickup_latitude']) & (df['pickup_latitude'] <= 42) & (38 <= df['dropoff_latitude']) & (df['dropoff_latitude'] <= 42)]

In [ ]:
df.shape

# Adding distance feature
***

In [ ]:
import geopy
from geopy.distance import geodesic as dist

In [ ]:
l1 = (40.721319, -73.844311)
l2 = (40.712278, -73.841610)
d = dist(l1, l2).miles
d

In [ ]:
def route_distance(row):
    d = dist((float(row['pickup_latitude']),float(row['pickup_longitude'])), (float(row['dropoff_latitude']),float(row['dropoff_longitude']))).miles
    return d

In [ ]:
def route_distance2(row):
    d = dist((row['pickup_latitude'],row['pickup_longitude']), (row['dropoff_latitude'],row['dropoff_longitude'])).miles
    return d

In [ ]:
df['distance'] = df.apply(lambda row: route_distance2(row), axis=1)

In [ ]:
def lat_diff(row):
    d = abs(row['pickup_latitude'] - row['dropoff_latitude'])
    return d

def long_diff(row):
    d = abs(row['pickup_longitude'] - row['dropoff_longitude'])
    return d

In [ ]:
df['lat_diff'] = df.apply(lambda row: lat_diff(row), axis=1)
df['long_diff'] = df.apply(lambda row: long_diff(row), axis=1)

In [ ]:
df.head()

# Date Time to different features
***

In [ ]:
def night(row):
    if (row['hour'] <= 6) or (row['hour'] >= 20):
        return 1
    else:
        return 0

In [ ]:
def morning_peak(row):
    if ((row['hour'] <= 10) and (row['hour'] >= 8)) and (row['weekday'] < 5):
        return 1
    else:
        return 0

In [ ]:
def evening_peak(row):
    if ((row['hour'] <= 19) and (row['hour'] >= 16)) and (row['weekday'] < 5):
        return 1
    else:
        return 0

In [ ]:
df['pickup_datetime'] =  pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S %Z')
df['month'] = df['pickup_datetime'].apply(lambda x: x.month)
df['day'] = df['pickup_datetime'].apply(lambda x: x.day)
df['hour'] = df['pickup_datetime'].apply(lambda x: x.hour)
df['weekday'] = df['pickup_datetime'].apply(lambda x: x.weekday())
df['pickup_datetime'] =  df['pickup_datetime'].apply(lambda x: str(x))
df['night'] = df.apply(lambda x: night(x), axis=1)
df['morning_peak'] = df.apply(lambda x: morning_peak(x), axis=1)
df['evening_peak'] = df.apply(lambda x: evening_peak(x), axis=1)

df = df.drop('pickup_datetime', axis=1)

In [ ]:
df.head()

In [ ]:
ddf = df.drop('key', axis=1)
ddf.head()

***
**Import for train and test**

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import feature_extraction, model_selection, naive_bayes, metrics, svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from IPython.display import Image
import warnings
import seaborn as sns
import string
import os

In [ ]:
from sklearn.metrics import classification_report
from sklearn import metrics

***
**Process data for train and test**

In [ ]:
train, test = train_test_split(ddf, test_size=0.2)
train.head()

In [ ]:
train.shape

In [ ]:
x_train = train[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','distance','month','day','hour','weekday','night','morning_peak','evening_peak']]
y_train = train[['fare_amount']]
x_test = test[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','distance','month','day','hour','weekday','night','morning_peak','evening_peak']]
y_test = test[['fare_amount']]

***
# ⚙ Custom Tools

In [ ]:
def get_train_test_xy(t_size):
    train, test = train_test_split(ddf, test_size=t_size)
    x_train = train[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','distance','month','day','hour','weekday','night','morning_peak','evening_peak']]
    y_train = train[['fare_amount']]
    x_test = test[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','distance','month','day','hour','weekday','night','morning_peak','evening_peak']]
    y_test = test[['fare_amount']]
    return x_train, y_train, x_test, y_test

In [ ]:
def add_error(row):
    val = ((row['fare_amount'] - row['prediction'])*100) / row['fare_amount']
    return val

In [ ]:
def regression_report(y_tst, y_prd, margin):
    y_tst['prediction']= y_prd.tolist()
    y_tst['error'] = y_tst.apply(lambda x: add_error(x), axis=1)
    pred_t = len(y_test['error'])
    pred_r = len(y_test[(y_test['error']>-margin) & (y_test['error']<margin)])
    pred_acc = (pred_r * 100)/pred_t
    return pred_t, pred_r, pred_acc

# Applying Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
clf = DecisionTreeRegressor()

x_train, y_train, x_test, y_test = get_train_test_xy(0.2)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,y_train)

y_pred = clf.predict(x_test)
pred_t, pred_r, pred_acc = regression_report(y_test, y_pred, 10)
print("Total: ", pred_t, ", Right within margin: ",pred_r, ", Accuracy: ", pred_acc, " %")

***
# Applying KNN Regressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
lst = []
for i in range(1,20):
    n = 5*i
    clf = KNeighborsRegressor(n_neighbors=n)

    x_train, y_train, x_test, y_test = get_train_test_xy(0.2)
    #Train the model using the training sets y_pred=clf.predict(X_test)
    clf.fit(x_train,y_train)

    y_pred = clf.predict(x_test)
    n = len(y_pred.tolist())
    y_pred = y_pred.reshape((n,)) 
    pred_t, pred_r, pred_acc = regression_report(y_test, y_pred, 10)
    lst.append(pred_acc)
    # print("Total: ", pred_t, ", Right within margin: ",pred_r, ", Accuracy: ", pred_acc, " %")
print(lst)

**(Linear Model) Ridge**

In [ ]:
from sklearn.linear_model import Ridge
clf = Ridge()

x_train, y_train, x_test, y_test = get_train_test_xy(0.2)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,y_train)

y_pred = clf.predict(x_test)
n = len(y_pred.tolist())
y_pred = y_pred.reshape((n,))
pred_t, pred_r, pred_acc = regression_report(y_test, y_pred, 10)
print("Total: ", pred_t, ", Right within margin: ",pred_r, ", Accuracy: ", pred_acc, " %")

**LinearRegression**

In [ ]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()

x_train, y_train, x_test, y_test = get_train_test_xy(0.2)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,y_train)

y_pred = clf.predict(x_test)
y_pred = y_pred.reshape((n,))
pred_t, pred_r, pred_acc = regression_report(y_test, y_pred, 10)
print("Total: ", pred_t, ", Right within margin: ",pred_r, ", Accuracy: ", pred_acc, " %")